In [ ]:
import importlib
import itertools
import urllib
import json
from IPython.display import display
import requests
import pandas as pd
import os

from utils import notebook_util
from prot_db.prot_sources.mgnify import mgnify_api, scrape_mgnify

notebook_util.disp_notebook_full_width()

importlib.reload(mgnify_api)
importlib.reload(scrape_mgnify)

In [ ]:
with open('/GeneGraphDB/data/scraper_debug/analysis.json', 'r') as f:
    analysis_json = json.load(f)
    
analysis_json

download_link = analysis_json['relationships']['downloads']['links']['related']

for download in mgnify_api.iterate_mgnify_url(download_link):
    label = download["attributes"]["description"]["label"]
    link = download["links"]["self"]
    
#     print(label)
#     print(link)
#     print("----")

    if label in scrape_mgnify.CDS_DOWNLOAD_DESCRIPTION_LABELS:
        display(download)


In [ ]:
os.path.basename(link)

In [ ]:
analysis_json

In [ ]:
df = pd.read_csv('/GeneGraphDB/data/scraper_debug/combined_annotations.gff', header=None, sep="\t", skiprows=1)
df.columns = ["seqid", "source", "other", "start", "end", "score", "strand", "phase", "attributes"]
df

In [ ]:
def parse_attributes(attributes):
    parsed = {}
    for entry in attributes.split(";"):
        key, value = entry.split("=")
        parsed[key] = value
    return parsed

parse_attributes(df.iloc[5]["attributes"])


In [ ]:
filtered = df[df["seqid"].str.startswith("ERZ1745275.25-NODE-25-length-45504-cov-18.102290")]
filtered = filtered[filtered["start"] == 3416]
filtered


In [ ]:
(df["start"] == 2).shape

In [ ]:
# Iterate over all MGNIFY studies  (~3750 studies in April 2020)

for study in mgnify_api.get_studies(start_page=148):
    print(study['id'])

study

In [ ]:
pd.read_csv('/GeneGraphDB/data/scraper_debug/diamond_head.tsv', sep="\t")

In [ ]:
for analysis in itertools.islice(mgnify_api.get_analyses('MGYS00002045'), 0, 51):
    print(analysis['id'])

analysis

In [ ]:
# test single sample id works
# sample_to_analysis = get_sample_to_analysis('MGYS00002045', "SRS373050")

sample_to_analysis = scrape_mgnify.get_sample_to_analysis('MGYS00002263')

In [ ]:
{k: v['id'] for k,v in sample_to_analysis.items()}

In [ ]:
analysis = list(sample_to_analysis.values())[0]

downloads_link = analysis['relationships']['downloads']['links']['related']
print(downloads_link)

# downloads_link = "https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00153401/downloads"

TGT_DOWNLOAD_DESCRIPTION_LABELS = [
    'Predicted CDS with annotation',
    'Predicted CDS without annotation'
]

for download in mgnify_api.iterate_mgnify_url(downloads_link):
    idx = 0
    if download['attributes']['description']['label'] in TGT_DOWNLOAD_DESCRIPTION_LABELS:
        display(download)
        
        src_url = download['links']['self']
        tgt_path = f"/GeneGraphDB/data/{download['attributes']['alias']}"
        
        urllib.request.urlretrieve(src_url, tgt_path)
        print("*****************")
        
    


In [ ]:
sample_generator = mgnify_api.get_samples('MGYS00000596')
for sample in itertools.islice(sample_generator, 0, 26):
    print(sample['id'])

sample

In [ ]:
# run_generator = get_runs(runs_url = sample['relationships']['runs']['links']['related'])
run_generator = mgnify_api.get_runs('SRS414107')
for run in run_generator:
    display(run)
    print("***************")

In [ ]:
# scratchwork from Thomas

def get_go_annotation(analysis, enarun):
    url = "https://www.ebi.ac.uk/metagenomics/api/v1/analyses/%s/file/%s_MERGED_FASTQ_GO_slim.csv" %(analysis, enarun)  
    s=requests.get(url).content
    return pd.read_csv(io.StringIO(s.decode('utf-8')), header=None, names=["tag", "desc", "", "i"])

def download_orf(analysis, enarun, target_path):
    # TODO extend for splitted files
    url = "https://www.ebi.ac.uk/metagenomics/api/v1/analyses/%s/file/%s_MERGED_FASTQ_ORF_unannotated.ffn.gz" %(analysis, enarun)  
    urllib.request.urlretrieve(url, target_path)
    
    
ANALYSIS_ID = 'MGYA00153401'


get_go_annotation()

https://www.ebi.ac.uk/metagenomics/api/v1/studies/MGYS00002045/pipelines/4.1/file/ERP104209_GO-slim_abundances_v4.1.tsv